# Tool 기반 투자 분석 Agent (ReAct 패턴)

## 주요 기능:
- ✅ **LangChain Tool 프레임워크** 활용
- ✅ **ReAct 패턴** (Reasoning + Acting)
- ✅ **4개의 투자 분석 도구**
- ✅ **LangGraph prebuilt agent** 사용
- ✅ **자동 도구 선택 및 실행**

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

## 1. 투자 분석 도구 가져오기

In [ ]:
import sys
sys.path.append('..')

from python.models.tools import (
    AVAILABLE_TOOLS,
    search_web,
    get_stock_price,
    calculate_moving_average,
    get_company_info,
    ToolAgentState
)

print(f"✅ {len(AVAILABLE_TOOLS)}개의 도구 로드 완료:")
for t in AVAILABLE_TOOLS:
    print(f"  - {t.name}: {t.description}")

## 2. ReAct Agent 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 시스템 프롬프트
system_prompt = """당신은 전문 주식 투자 분석가입니다.

**역할**:
- 사용자의 투자 관련 질문에 데이터 기반으로 답변
- 필요한 경우 제공된 도구를 사용하여 정보 수집
- 실시간 주가, 기업 정보, 뉴스 등을 활용한 분석

**사용 가능한 도구**:
1. search_web: 웹에서 최신 뉴스 및 정보 검색
2. get_stock_price: 특정 주식의 가격 정보 조회
3. calculate_moving_average: 기술적 분석 (이동평균선)
4. get_company_info: 기업 기본 정보 및 재무 지표

**답변 원칙**:
- 구체적인 데이터와 출처를 제시
- 불확실한 정보는 명시
- 투자 결정은 사용자의 몫임을 강조
"""

# ReAct Agent 생성
agent = create_react_agent(
    llm,
    AVAILABLE_TOOLS,
    state_modifier=system_prompt
)

print("✅ ReAct Agent 생성 완료")
print(f"   도구 개수: {len(AVAILABLE_TOOLS)}")
print(f"   LLM 모델: {llm.model_name}")

## 3. 그래프 구조 시각화

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"그래프 시각화 실패: {e}")
    print("\n텍스트 구조:")
    print("START → __start__ → agent → tools → agent → __end__ → END")

## 4. Agent 실행 헬퍼 함수

In [ ]:
def run_agent(query: str, verbose: bool = True):
    """
    Agent를 실행하고 결과를 출력합니다.
    
    Args:
        query: 사용자 질문
        verbose: 실행 과정 출력 여부
    """
    print(f"\n{'='*70}")
    print(f"질문: {query}")
    print(f"{'='*70}\n")
    
    messages = [{"role": "user", "content": query}]
    
    for chunk in agent.stream({"messages": messages}, stream_mode="values"):
        if verbose:
            last_message = chunk["messages"][-1]
            
            if hasattr(last_message, 'content') and last_message.content:
                print(f"[{last_message.__class__.__name__}]")
                print(last_message.content)
                print()
            elif hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                for tc in last_message.tool_calls:
                    print(f"🔧 도구 호출: {tc['name']}")
                    print(f"   입력: {tc['args']}")
                print()
    
    # 최종 답변
    final_message = chunk["messages"][-1]
    print(f"\n{'='*70}")
    print("최종 답변:")
    print(f"{'='*70}")
    print(final_message.content)
    print(f"{'='*70}\n")
    
    return chunk

print("✅ 헬퍼 함수 정의 완료")

## 5. 테스트 1: 간단한 주가 조회

In [ ]:
result1 = run_agent("삼성전자 현재 주가 알려줘")

## 6. 테스트 2: 복합 분석 (여러 도구 사용)

ReAct 패턴에 따라 Agent가 여러 도구를 순차적으로 사용합니다.

In [ ]:
result2 = run_agent(
    "삼성전자(005930.KS)의 현재 주가와 20일 이동평균을 비교하고, "
    "최근 관련 뉴스도 검색해서 투자 의견을 제시해줘"
)

## 7. 테스트 3: 미국 주식 분석

In [ ]:
result3 = run_agent(
    "Apple(AAPL) 주식의 기업 정보와 최근 3개월 주가 추이를 분석해줘"
)

## 8. 불변 속성 검증 예시

Agent 상태의 불변 속성을 런타임에 검증할 수 있습니다.

In [ ]:
from python.models.tools import ToolHistory, ToolExecution

# 초기 상태 생성
state = ToolAgentState.initial_state("테스트 질문", max_calls=5)

# 도구 실행 시뮬레이션
exec1 = ToolExecution(
    tool_name="search_web",
    arguments={"query": "test"},
    result="테스트 결과",
    call_id="call_1"
)
state.tool_history.add_execution(exec1)
state.current_iteration += 1

# 불변 속성 검증
print("불변 속성 검증:")
print(f"  도구 호출 횟수 제한: {state.tool_history.total_calls <= state.max_tool_calls}")
print(f"  히스토리 일관성: {len(state.tool_history.executions) == state.tool_history.total_calls}")
print(f"  전체 검증: {state.verify_invariants()}")
print(f"\n현재 상태:")
print(f"  도구 호출 횟수: {state.tool_history.total_calls}/{state.max_tool_calls}")
print(f"  더 호출 가능: {state.can_call_more_tools()}")

## 9. 연습 문제

### 문제 1: RSI 지표 도구 추가

`python/models/tools.py`에 새로운 도구를 추가하세요.

```python
@tool
def calculate_rsi(
    ticker: Annotated[str, "주식 티커 심볼"],
    period: Annotated[int, "RSI 계산 기간"] = 14
) -> str:
    """
    RSI 지표를 계산합니다.
    70 이상: 과매수, 30 이하: 과매도
    """
    # TODO: 구현하세요
    pass
```

**힌트**:
- `yfinance`로 과거 데이터 가져오기
- RSI 공식: RSI = 100 - (100 / (1 + RS)), RS = 평균 상승폭 / 평균 하락폭
- `AVAILABLE_TOOLS` 리스트에 추가하기

### 문제 2: 포트폴리오 분석 도구

여러 주식을 입력받아 포트폴리오 수익률을 계산하는 도구를 만드세요.

```python
@tool
def analyze_portfolio(
    tickers: Annotated[str, "쉼표로 구분된 티커 목록 (예: AAPL,MSFT,GOOGL)"],
    weights: Annotated[str, "쉼표로 구분된 비중 (예: 0.4,0.3,0.3)"] = None
) -> str:
    """
    포트폴리오의 수익률과 리스크를 분석합니다.
    """
    # TODO: 구현하세요
    pass
```

### 문제 3: 시스템 프롬프트 개선

Agent의 시스템 프롬프트를 수정하여 더 나은 투자 조언을 제공하도록 만드세요.

**고려사항**:
- 리스크 경고 추가
- 투자 기간별 조언 (단기/중기/장기)
- 시장 상황에 따른 전략

### 문제 4: 에러 처리 개선

도구 실행 중 발생할 수 있는 에러를 더 잘 처리하도록 코드를 개선하세요.

**예시**:
- 잘못된 티커 심볼
- 네트워크 오류
- API 제한 초과